In [1]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import matplotlib as mpl
import gc
import os

from model import (
    LeafcutterAntsFungiMutualismModel, AntAgent, Plant, Nest, Fungus,
    AntWorkerState, track_forager_fitness
)
from model.pheromone import Pheromone
from model.ant_agent import AntWorkerState

from batchrunner import BatchRunnerMP

# plot settings
mpl.rcParams["figure.dpi"] = 200
mpl.rcParams["legend.fontsize"] = 12
mpl.rcParams["font.size"] = 8

TEX_PLOTS = True

if TEX_PLOTS:
    plt.rc("text", usetex=True)
    plt.rc("text.latex", preamble=r"""
     \usepackage{times}
     \usepackage{mathptmx}""")
else:
    plt.rc("text", usetex=False)
    
plt.rc("font", family="serif")


def visualize_agent(agent, x, y):
    n_ants = 0
    n_foragers = 0
    n_caretakers = 0
    
    if agent is None:
        return
    
    MS = 2.7

    if isinstance(agent, AntAgent):
        MS = 2.1
        plt.plot(x, y, ".", ms=MS+2, color="tab:brown", zorder=5)
        if agent.has_leaf:
            plt.plot(x, y, "o", color="tab:green", markersize=1, zorder=6)
            
        if agent.state is AntWorkerState.CARETAKING:
            n_caretakers += 1
        else:
            n_foragers +=1
        n_ants += 1
    elif isinstance(agent, Plant):
        ratio = agent.num_leaves/agent.initial_num_leaves
        ms = min(MS*(ratio+0.1), MS)
        plt.plot(x, y, "^", color="tab:green", markersize=ms, zorder=4)
    elif isinstance(agent, Nest):
        plt.plot(x, y, "s", color="tab:orange", zorder=0)
    elif isinstance(agent, Fungus):
        pass
    elif isinstance(agent, Pheromone):
        MS = 4.5
        ratio = agent.lifespan/agent.initial_lifespan
        ms = min(MS*(ratio+0.1), MS)
        plt.plot(x, y, "o", ms=ms,  mfc="none", color="tab:blue", zorder=-1, markeredgewidth=0.5)
    else:
        print(f"Not yet visualized agent {agent.__class__}!")
        
    return n_ants, n_foragers, n_caretakers


def visualize_grid(grid, i):
    filename = f"figures/gif_frames/{i}.png"
    n_ants = 0
    n_foragers = 0
    n_caretakers = 0

    plt.figure(figsize=(5, 5))
    for agents, x, y in grid.coord_iter():
        for agent in agents:
            if isinstance(agent, AntAgent):
                if agent.state is AntWorkerState.CARETAKING:
                    n_caretakers += 1
                else:
                    n_foragers +=1
                n_ants += 1

            visualize_agent(agent, x, y)

    if n_ants == 0:
        plt.title("Population extinct")
    else:
        frac_foragers = n_foragers/n_ants
        plt.title(f"Population size: {n_ants} ({frac_foragers:.2f} foragers, {1-frac_foragers:.2f} caretakers)")
    
    plt.xlim(-0.5, 50.5)
    plt.ylim(-0.5, 50.5) 
    #plt.axis('off')
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.savefig(filename, facecolor='white', transparent=False)
    
    # https://stackoverflow.com/questions/7101404/how-can-i-release-memory-after-creating-matplotlib-figures
    plt.clf()
    plt.close()
    gc.collect()

    
def run_model(args):
    m = LeafcutterAntsFungiMutualismModel(collect_data=False, seed=args["seed"])
    
    np.random.seed(args["seed"])
    
    i = 0

    while m.running and m.schedule.steps < args["time_steps"]:
        m.step()
        visualize_grid(m.grid, i)
        i += 1

    return m

In [2]:
# collect frames for gif
kwargs = {"time_steps": 500, "seed": 1212}
model = run_model(kwargs)

In [3]:
# create gif
files = [f"figures/gif_frames/{i}.png" for i in range(0, kwargs["time_steps"])]
with imageio.get_writer("figures/demo.gif", mode="I", fps=20) as writer:
    for filename in files:
        image = imageio.imread(filename)
        writer.append_data(image)

In [4]:
for filename in set(files):
    os.remove(filename)